In [0]:
%pip install --no-dependencies tensorflow-recommenders tensorflow-datasets tensorflow_ranking scann pydot

Python interpreter will be restarted.
Requirement already satisfied: tensorflow-recommenders in /local_disk0/.ephemeral_nfs/envs/pythonEnv-cbb2f719-6e8b-4c74-8b9a-89debba7c324/lib/python3.8/site-packages (0.6.0)
Requirement already satisfied: tensorflow-datasets in /local_disk0/.ephemeral_nfs/envs/pythonEnv-cbb2f719-6e8b-4c74-8b9a-89debba7c324/lib/python3.8/site-packages (4.4.0)
Requirement already satisfied: tensorflow_ranking in /local_disk0/.ephemeral_nfs/envs/pythonEnv-cbb2f719-6e8b-4c74-8b9a-89debba7c324/lib/python3.8/site-packages (0.5.0)
Requirement already satisfied: scann in /local_disk0/.ephemeral_nfs/envs/pythonEnv-cbb2f719-6e8b-4c74-8b9a-89debba7c324/lib/python3.8/site-packages (1.2.4)
Requirement already satisfied: pydot in /databricks/python3/lib/python3.8/site-packages (1.4.2)
Python interpreter will be restarted.

In [0]:
%load_ext tensorboard

In [0]:
import mlflow
import tensorflow as tf
import tensorflow_recommenders as tfrs
from spark_tensorflow_distributor import MirroredStrategyRunner

print([tf.__version__, tf.config.list_physical_devices('GPU')])

['2.7.0', []]

In [0]:
dbutils.widgets.text("num_epochs", "3")
NUM_EPOCHS = int(dbutils.widgets.get("num_epochs"))

dbutils.widgets.text("batch_size", "20000")
BATCH_SIZE = int(dbutils.widgets.get("batch_size"))

print(NUM_EPOCHS, BATCH_SIZE)

path_prefix = "/sais22_sample"
data_path = path_prefix + "/data"

best_model_path = f"/dbfs{path_prefix}/model/best"
chkpt_model_path = f"/dbfs{path_prefix}/model/chkpt"

# needed to access dbfs from python code and tf.data
get_python_path = (lambda path: f"/dbfs{path}")

print(path_prefix, get_python_path(path_prefix), sep="\n")

3 10000
/sais22_sample
/dbfs/sais22_sample

In [0]:
def get_num_workers():
  """
    source: https://stackoverflow.com/questions/28768642/getting-number-of-visible-nodes-in-pyspark
  """
  s = sc._jsc.sc().getExecutorMemoryStatus().keys()
  l = str(s).replace("Set(","").replace(")","").split(", ")

  d = set()
  for i in l:
      d.add(i.split(":")[0])
  return len(d) - 1

NUM_WORKERS = get_num_workers()
print(f"Number of workers: {NUM_WORKERS}")

# Assume the driver node and worker nodes have the same instance type.
NUM_GPUS = len(tf.config.list_logical_devices('GPU')) * NUM_WORKERS
NUM_CPUS = len(tf.config.list_logical_devices('CPU')) * NUM_WORKERS
USE_GPU = NUM_GPUS > 0
NUM_SLOTS =NUM_GPUS if USE_GPU else NUM_CPUS

print(f"num GPUs: {NUM_GPUS}, num CPUs: {NUM_CPUS}")
print(f"Use GPU: {USE_GPU}, num slots: {NUM_SLOTS}")

Number of workers: 6
num GPUs: 0, num CPUs: 6
Use GPU: False, num slots: 6

In [0]:
try:
    strategy = tf.distribute.MirroredStrategy()
except:
    strategy = tf.distribute.get_strategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

WARNING:tensorflow:There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of replicas: 1

In [0]:
def log_dict_to_mlflow(d, artifact_path=None):
  import json
  import tempfile
  import mlflow
  with tempfile.NamedTemporaryFile(mode='w', suffix='.json') as f:
    # Note the mode is 'w' so json could be dumped
    # Note the suffix is .txt so the UI will show the file
    json.dump(d, f)
    f.seek(0) # You cannot close the file as it will be removed. You have to move back to its head
    mlflow.log_artifact(f.name, artifact_path=artifact_path)
    
class MlFlowLogging(tf.keras.callbacks.Callback):
  import mlflow

  def __init__(self, artifact_name="model", *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.artifact_name = artifact_name

  def on_train_end(self, logs=None):
    super().on_train_end(logs)
#       mlflow.keras.log_model(self.model, f"{self.artifact_name}/final")

  def on_epoch_end(self, epoch, logs=None):
    super().on_epoch_end(epoch, logs)
    keys = list(logs.keys())
    for key in keys:
      if "Tensor" not in str(type(logs[key])):
        mlflow.log_metric(key, float(logs[key]), epoch)
        continue
      mlflow.log_metric(key, float(logs[key].numpy()), epoch)

In [0]:
def get_callbacks(is_chief, logdir, model_save_path, model_chkpt_path, monitor_metric="val_loss", early_stop_patience=3, reduce_lr_patience=1, reduce_lr_factor=0.5):
  import os
  from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
  import mlflow  
  
  reduce_lr = ReduceLROnPlateau(monitor=monitor_metric, factor=reduce_lr_factor, verbose=1,
                                min_delta=0.01, min_lr=1e-6, patience=reduce_lr_patience)
  early_stop = EarlyStopping(monitor=monitor_metric, patience=early_stop_patience, min_delta=0.01,
                               verbose=1)
  
  best_model_save_callback = ModelCheckpoint(
      filepath=model_save_path,
      monitor=monitor_metric,
      save_best_only=True,
      )
  
  model_chkpt_save_callback_tmp = ModelCheckpoint(
    filepath="some_random_path_chkpt",
    monitor=monitor_metric,  
    save_weights_only=True
    )
  
  model_chkpt_save_callback = ModelCheckpoint(
    filepath=os.path.join(model_chkpt_path, "ckpt_{epoch}"),
    monitor=monitor_metric, 
    save_weights_only=True,
    )
  
  best_model_save_callback_tmp = ModelCheckpoint(
    filepath="some_random_path",
    monitor=monitor_metric,
    save_best_only=True
    )
  
  mlflow_logging = MlFlowLogging()
  
  
  profile_batch = [1,2]
  tensorboard = TensorBoard(logdir, profile_batch=profile_batch)
  tensorboard_worker_tmp = TensorBoard("tmp_logs", profile_batch=profile_batch)
  
  callbacks = [
    reduce_lr, 
    early_stop
  ]
  
  if is_chief:
    callbacks += [
      best_model_save_callback,
      model_chkpt_save_callback,
      tensorboard,
      mlflow_logging
     ]   
    
  else:
    callbacks += [
      best_model_save_callback_tmp,
      model_chkpt_save_callback_tmp,
      tensorboard_worker_tmp
     ]
    
  return callbacks

### Process data using Spark

In [0]:
def create_tfrecords_using_spark():
  dbfs_dir = '/databricks-datasets/cs110x/ml-20m/data-001'

  ratings_filename = dbfs_dir + '/ratings.csv' 
  movies_filename = dbfs_dir + '/movies.csv'

  ratings = spark.read.option("header", "true").csv(ratings_filename).selectExpr("cast(movieId as string) movie_id", "cast(userId as string) user_id")
  movies = spark.read.option("header", "true").csv(movies_filename).selectExpr("cast(movieId as string) movie_id", "title")

  train, test = ratings.join(movies, ["movie_id"]).select("movie_id", "user_id", "title").distinct().randomSplit([0.8, 0.2], seed=123)

  #limiting counts for demo
  train = train.limit(80000)
  test = test.limit(20000)

  sampled_df = train.unionByName(test)

  users = sampled_df.select("user_id").distinct()
  print(users.count())

  print(train.count())
  print(test.count())

  train.repartition(10).write.format("tfrecords").mode("overwrite").save(f"{data_path}/train")
  test.repartition(5).write.format("tfrecords").mode("overwrite").save(f"{data_path}/test")
  users.repartition(3).write.format("tfrecords").mode("overwrite").save(f"{data_path}/users")
  #limit to only movies in the sample
  movies.join(sampled_df.select("movie_id").distinct(), ["movie_id"]).repartition(3).write.format("tfrecords").mode("overwrite").save(f"{data_path}/movies")
  
create_tfrecords_using_spark()

52384
80000
20000

### Reading data as tf dataset

Helpers for tf.data

now we are ready to load the datasets

In [0]:
def make_datasets(train_shuffle_buffer=100000, test_shuffle_buffer=20000, distributed=False):
  import os
  import datetime
  import mlflow

  import numpy as np
  import pandas as pd
  import tensorflow as tf
  import tensorflow.keras.layers as L
  
  from tensorflow import keras
  from typing import Dict, Text
  
  def generate_filepaths(typ: str) -> [str]:  
    """
    this method creates a list of file paths, to be read by tf.data
    """
    import os
    
    out = []
    for base,_,files in os.walk(os.path.join("/dbfs" + data_path, typ)):
      out = [os.path.join(base, file) for file in files if not file.startswith("_")]
      return out

  def read_movies_tfrecord(example):
    tfrecord_format = (
        {
          "movie_id": tf.io.FixedLenFeature([], tf.string),
          "title":tf.io.FixedLenFeature([], tf.string)
        }
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    return example

  def read_interaction_tfrecord(example):
    tfrecord_format = (
        {
            "movie_id": tf.io.FixedLenFeature([], tf.string),
            "user_id": tf.io.FixedLenFeature([], tf.string),
            "title":tf.io.FixedLenFeature([], tf.string)
        }
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    return example
  
  def create_lookup_for(col_name, dataset):
    lookup = L.StringLookup(num_oov_indices=1, name=f"{col_name}_lookup")
    lookup.adapt(dataset.map(lambda r: r[col_name]).batch(500))
    print(f"Vocabulary size for {col_name} lookup: {lookup.vocabulary_size()}")
    return lookup
  
  def make_distributed_dataset(dataset, shuffle=False):
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA

    return dataset.with_options(options).batch(BATCH_SIZE, drop_remainder=True)
  
  movies_ds = tf.data.TFRecordDataset(filenames=generate_filepaths("movies")).map(read_movies_tfrecord)
  train_ds = tf.data.TFRecordDataset(filenames=generate_filepaths("train")).map(read_interaction_tfrecord)
  test_ds = tf.data.TFRecordDataset(filenames=generate_filepaths("test")).map(read_interaction_tfrecord)

  
  # verifying if the datta was read properly
  # always good to test a batch so we can capture issues
  # if you see NaN issues here, it would be mostly some form of data type issue
  for row in movies_ds.batch(10).take(1):
    print(row)
  
  for row in train_ds.batch(10).take(1):
    print(row)

  for row in test_ds.batch(10).take(1):
    print(row)
    
  user_cols = ["user_id"]
  movie_cols = ["title"]
  
  lookups = {**{col: create_lookup_for(col, train_ds) for col in user_cols}, **{col: create_lookup_for(col, movies_ds) for col in movie_cols}}
  

  if distributed:
    return make_distributed_dataset(train_ds), make_distributed_dataset(test_ds), movies_ds, lookups
  return train_ds.batch(BATCH_SIZE, drop_remainder=True), test_ds.batch(BATCH_SIZE, drop_remainder=True), movies_ds, lookups

In [0]:
train_ds, test_ds, movies_ds, lookups = make_datasets()
lookups

{'movie_id': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'116758', b'1997', b'103235', b'1963', b'100579', b'100344',
 b'201', b'100843', b'129335', b'105213'], dtype=object)>, 'title': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'Death Racers (2008)', b'Exorcist, The (1973)',
 b'Best Offer, The (Migliore offerta, La) (2013)',
 b'Take the Money and Run (1969)',
 b'Universal Soldier: Day of Reckoning (2012)', b'No (2012)',
 b'Three Wishes (1995)', b'Oh Boy (A Coffee in Berlin) (2012)',
 b'A Summer in La Goulette (1996)', b'Don Jon (2013)'], dtype=object)>}
{'movie_id': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'1882', b'1215', b'1982', b'1387', b'196', b'1210', b'165',
 b'1270', b'1610', b'1291'], dtype=object)>, 'title': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'Godzilla (1998)', b'Army of Darkness (1993)',
 b'Halloween (1978)', b'Jaws (1975)', b'Species (1995)',
 b'Star Wars: Episode VI - Return of the Jedi (1983)',
 b'Die Hard: With a Vengeance (1995)', b'Back to the Future (1985)',
 b'Hunt for Red October, The (1990)',
 b'Indiana Jones and the Last Crusade (1989)'], dtype=object)>, 'user_id': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'6262', b'95566', b'129360', b'86359', b'46853', b'113849',
 b'26857', b'17539', b'91379', b'71621'], dtype=object)>}
{'movie_id': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'1608', b'1917', b'1227', b'1305', b'1270', b'1270', b'196',
 b'1658', b'1444', b'193'], dtype=object)>, 'title': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'Air Force One (1997)', b'Armageddon (1998)',
 b'Once Upon a Time in America (1984)', b'Paris, Texas (1984)',
 b'Back to the Future (1985)', b'Back to the Future (1985)',
 b'Species (1995)', b'Life Less Ordinary, A (1997)',
 b'Guantanamera (1994)', b'Showgirls (1995)'], dtype=object)>, 'user_id': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'80918', b'65084', b'109750', b'110408', b'108582', b'132010',
 b'125081', b'64816', b'115073', b'81544'], dtype=object)>}
Vocabulary size for user_id lookup: 45963
Vocabulary size for title lookup: 1703
Out[16]: {'user_id': <keras.layers.preprocessing.string_lookup.StringLookup at 0x7f11542b1d90>,
 'title': <keras.layers.preprocessing.string_lookup.StringLookup at 0x7f1154299760>}

In [0]:
from typing import Dict, List, Text
import os

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model, movies):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
      metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.map(lambda r: r["title"]).batch(1000).map(movie_model)
      ))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings, compute_metrics=not training)
 
  def summary(self, *args, **kwargs):
    self.user_model.summary(*args, **kwargs)
    self.movie_model.summary(*args, **kwargs)
    
  def save(self, location, **kwargs):
    print(f"save to location {location}")
    self.user_model.save(os.path.join(location, 'user'), **kwargs)
    self.movie_model.save(os.path.join(location, 'movie'), **kwargs)
    
  def save_weights(self, location, **kwargs):
    print(f"save_weights to location {location}")
    self.user_model.save_weights(os.path.join(location, 'user'), **kwargs)
    self.movie_model.save_weights(os.path.join(location, 'movie'), **kwargs)
    
  def load_weights(self, location, **kwargs):
    self.user_model.load_weights(os.path.join(location, 'user'), **kwargs)
    self.movie_model.load_weights(os.path.join(location, 'movie'), **kwargs)

In [0]:
def make_model(lookups, movies_ds, embedding_dimension=8):
  import tensorflow as tf
  
  user_lookup = lookups["user_id"]
  movie_title_lookup = lookups["title"]
  user_model = tf.keras.Sequential([
    user_lookup,
    tf.keras.layers.Embedding(user_lookup.vocabulary_size(), embedding_dimension)
  ])

  movie_model = tf.keras.Sequential([
    movie_title_lookup,
    tf.keras.layers.Embedding(movie_title_lookup.vocabulary_size(), embedding_dimension)
  ])

  model = MovielensModel(user_model, movie_model, movies_ds)
  model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))
  
  return model

In [0]:
model = make_model(lookups, movies_ds)

In [0]:
import datetime
experiment_log_dir = "/dbfs/tf_logs"
log_dir = experiment_log_dir + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [0]:
%tensorboard --logdir $experiment_log_dir

Launching TensorBoard...

Open in a new tab
 
 Note: TensorBoard is only available when this notebook remains attached to the cluster.

In [0]:
model.fit(train_ds, epochs=3, callbacks=get_callbacks(is_chief=True, logdir=log_dir, model_save_path=best_model_path, model_chkpt_path=chkpt_model_path))

Epoch 1/3
 1/Unknown - 18s 18s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 92103.2734 - regularization_loss: 0.0000e+00 - total_loss: 92103.2734 2/Unknown - 42s 23s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 92103.3086 - regularization_loss: 0.0000e+00 - total_loss: 92103.3086 3/Unknown - 43s 12s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 92103.3958 - regularization_loss: 0.0000e+00 - total_loss: 92103.3958 4/Unknown - 44s 9s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 92103.3789 - regularization_loss: 0.0000e+00 - total_loss: 92103.3789  5/Unknown - 45s 7s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 92103.3797 - regularization_loss: 0.0000e+00 - total_loss: 92103.3797

Distributed Runner

In [0]:
def is_node_chief():
  import os
  import json 
  
  is_chief = True
  if 'TF_CONFIG' in os.environ:
    tf_config = json.loads(os.environ['TF_CONFIG'])
    print(tf_config)
    print(tf_config['task'])
    node_index = tf_config['task']['index']
    is_chief = node_index == 0
    print(f"Node Index: {node_index}, Is Chief: {is_chief}")
    return is_chief

In [0]:
def train():  
  import os
  import mlflow
  import datetime
  
  import tensorflow as tf
  
  print([tf.__version__, tf.config.list_physical_devices('GPU')])
#   monitor_metric = "val_loss"
  logdir = os.path.join(experiment_log_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  
  is_chief = is_node_chief()

  train_ds, test_ds, movies_ds, lookups = make_datasets(distributed=True)
  print(lookups)
  
  multi_worker_model = make_model(lookups, movies_ds)
  
  if is_chief:
    mlflow.log_param("num_workers", NUM_SLOTS)
    mlflow.log_param("batch_size", BATCH_SIZE)

#     mlflow.log_param("model_embedding_size", MODEL_EMBEDDING_SIZE)
#     mlflow.log_param("learning_rate", LR)
#     mlflow.log_param("num_epochs", NUM_EPOCHS)
    
    for k,v in lookups.items():
      if v.dtype == "string":
        log_dict_to_mlflow(v.get_vocabulary(), f"inputs/lookups/{k}")


  history = multi_worker_model.fit(train_ds.prefetch(10), 
                                   epochs=int(NUM_EPOCHS),
                                   validation_data=test_ds.prefetch(10),
                                   callbacks=get_callbacks(is_chief, 
                                                           logdir=logdir,
                                                           model_save_path=best_model_path, 
                                                           model_chkpt_path=chkpt_model_path))

  if is_chief:
    for key in history.history.keys():
      [mlflow.log_metric(f"{key}_history", float(l), i) for i, l in enumerate(history.history[key])]
    mlflow.log_param("final_epochs", str(len(history.history[list(history.history.keys())[0]])))

In [0]:
# Use local_mode=True to test on driver node, False will run it on all workers
runner = MirroredStrategyRunner(num_slots=NUM_SLOTS, use_gpu=USE_GPU)
runner.run(train)

Doing CPU training...
Will run with 6 Spark tasks.
Distributed training in progress...
View Spark executor stderr logs to inspect training...
Training with 6 slots is complete!

Store model to MLFlow

In [0]:
%sh apt-get install -y graphviz

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
 fonts-liberation libann0 libcdt5 libcgraph6 libgts-0.7-5 libgts-bin libgvc6
 libgvpr2 liblab-gamut1 libpathplan4
Suggested packages:
 graphviz-doc
The following NEW packages will be installed:
 fonts-liberation graphviz libann0 libcdt5 libcgraph6 libgts-0.7-5 libgts-bin
 libgvc6 libgvpr2 liblab-gamut1 libpathplan4
0 upgraded, 11 newly installed, 0 to remove and 28 not upgraded.
Need to get 2701 kB of archives.
After this operation, 11.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 fonts-liberation all 1:1.07.4-11 [822 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libann0 amd64 1.1.2+doc-7build1 [26.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libcdt5 amd64 2.42.2-3build2 [18.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libcgraph6 amd64 2.42.2-3build2 [41.3 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/universe amd64 libgts-0.7-5 amd64 0.7.6+darcs121130-4 [150 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal/universe amd64 libpathplan4 amd64 2.42.2-3build2 [21.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal/universe amd64 libgvc6 amd64 2.42.2-3build2 [647 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal/universe amd64 libgvpr2 amd64 2.42.2-3build2 [167 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal/universe amd64 liblab-gamut1 amd64 2.42.2-3build2 [177 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal/universe amd64 graphviz amd64 2.42.2-3build2 [590 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal/universe amd64 libgts-bin amd64 0.7.6+darcs121130-4 [41.3 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 2701 kB in 1s (3145 kB/s)
Selecting previously unselected package fonts-liberation.
(Reading database ... 
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 95825 files and directories currently installed.)
Preparing to unpack .../00-fonts-liberation_1%3a1.07.4-11_all.deb ...
Unpacking fonts-liberation (1:1.07.4-11) ...
Selecting previously unselected package libann0.
Preparing to unpack .../01-libann0_1.1.2+doc-7build1_amd64.deb ...
Unpacking libann0 (1.1.2+doc-7build1) ...
Selecting previously unselected package libcdt5:amd64.
Preparing to unpack .../02-libcdt5_2.42.2-3build2_amd64.deb ...
Unpacking libcdt5:amd64 (2.42.2-3build2) ...
Selecting previously unselected package libcgraph6:amd64.
Preparing to unpack .../03-libcgraph6_2.42.2-3build2_amd64.deb ...
Unpacking libcgraph6:amd64 (2.42.2-3build2) ...
Selecting previously unselected package libgts-0.7-5:amd64.
Preparing to unpack .../04-libgts-0.7-5_0.7.6+darcs121130-4_amd64.deb ...
Unpacking libgts-0.7-5:amd64 (0.7.6+darcs121130-4) ...
Selecting previously unselected package libpathplan4:amd64.
Preparing to unpack .../05-libpathplan4_2.42.2-3build2_amd64.deb ...
Unpacking libpathplan4:amd64 (2.42.2-3build2) ...
Selecting previously unselected package libgvc6.
Preparing to unpack .../06-libgvc6_2.42.2-3build2_amd64.deb ...
Unpacking libgvc6 (2.42.2-3build2) ...
Selecting previously unselected package libgvpr2:amd64.
Preparing to unpack .../07-libgvpr2_2.42.2-3build2_amd64.deb ...
Unpacking libgvpr2:amd64 (2.42.2-3build2) ...
Selecting previously unselected package liblab-gamut1:amd64.
Preparing to unpack .../08-liblab-gamut1_2.42.2-3build2_amd64.deb ...
Unpacking liblab-gamut1:amd64 (2.42.2-3build2) ...

In [0]:
%scala
// Used only for local runs, workaround to get notebook experiment name
dbutils.widgets.text("notebook", dbutils.notebook.getContext().notebookPath.get)

In [0]:
exp_id = mlflow.get_experiment_by_name(dbutils.widgets.get("notebook")).experiment_id
run_id = mlflow.list_run_infos(exp_id)[0].run_id
run_id

Out[52]: '7d402175957b482b8b8b373d6b541bd4'

create a lookup model

In [0]:
import os

k = 200
movie_model = tf.keras.models.load_model(os.path.join(best_model_path, "movie"))
user_model = tf.keras.models.load_model(os.path.join(best_model_path, "user"))

index = tfrs.layers.factorized_top_k.ScaNN(query_model=user_model, num_reordering_candidates=1000, k=k, name="scann_lookup")

index.index_from_dataset(
  tf.data.Dataset.zip((movies_ds.map(lambda r: r["movie_id"]).batch(100), 
                       movies_ds.map(lambda r: r["title"]).batch(100).map(movie_model)))
)

for row in test_ds.unbatch().batch(10).take(1):
  print(index(row["user_id"]))

index.save(
      f"{best_model_path}/lookup",
      overwrite=True,
      options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"])
    )

WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
(<tf.Tensor: shape=(10, 156), dtype=float32, numpy=
array([[0.00707278, 0.00703982, 0.00699814, ..., nan, nan,
 nan],
 [0.00628913, 0.00561393, 0.00549097, ..., nan, nan,
 nan],
 [0.00707278, 0.00703982, 0.00699814, ..., nan, nan,
 nan],
 ...,
 [0.00707278, 0.00703982, 0.00699814, ..., nan, nan,
 nan],
 [0.00707278, 0.00703982, 0.00699814, ..., nan, nan,
 nan],
 [0.00767101, 0.00748365, 0.00724999, ..., nan, nan,
 nan]], dtype=float32)>, <tf.Tensor: shape=(10, 156), dtype=string, numpy=
array([[b'115109', b'1651', b'1164', ..., b'116758', b'116758',
 b'116758'],
 [b'117456', b'117368', b'119139', ..., b'116758', b'116758',
 b'116758'],
 [b'115109', b'1651', b'1164', ..., b'116758', b'116758',
 b'116758'],
 ...,
 [b'115109', b'1651', b'1164', ..., b'116758', b'116758',
 b'116758'],
 [b'115109', b'1651', b'1164', ..., b'116758', b'116758',
 b'116758'],
 [b'1140', b'1124', b'114762', ..., b'116758', b'116758',
 b'116758']], dtype=object)>)
WARNING:absl:Found untraced functions such as query_with_exclusions while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: /dbfs/sais22_sample/model/best/lookup/assets
INFO:tensorflow:Assets written to: /dbfs/sais22_sample/model/best/lookup/assets

In [0]:
### Verify if we can load the saved lookup model and get recommendations ###
lookup_model = tf.saved_model.load(f"{base_download_path}/models/saved_model/lookup")
for row in test_ds.unbatch().batch(10).take(1):
  print(lookup_model(row["user_id"]))

(<tf.Tensor: shape=(10, 156), dtype=float32, numpy=
array([[0.00707278, 0.00703982, 0.00699814, ..., nan, nan,
 nan],
 [0.00628913, 0.00561393, 0.00549097, ..., nan, nan,
 nan],
 [0.00707278, 0.00703982, 0.00699814, ..., nan, nan,
 nan],
 ...,
 [0.00707278, 0.00703982, 0.00699814, ..., nan, nan,
 nan],
 [0.00707278, 0.00703982, 0.00699814, ..., nan, nan,
 nan],
 [0.00767101, 0.00748365, 0.00724999, ..., nan, nan,
 nan]], dtype=float32)>, <tf.Tensor: shape=(10, 156), dtype=string, numpy=
array([[b'115109', b'1651', b'1164', ..., b'116758', b'116758',
 b'116758'],
 [b'117456', b'117368', b'119139', ..., b'116758', b'116758',
 b'116758'],
 [b'115109', b'1651', b'1164', ..., b'116758', b'116758',
 b'116758'],
 ...,
 [b'115109', b'1651', b'1164', ..., b'116758', b'116758',
 b'116758'],
 [b'115109', b'1651', b'1164', ..., b'116758', b'116758',
 b'116758'],
 [b'1140', b'1124', b'114762', ..., b'116758', b'116758',
 b'116758']], dtype=object)>)

In [0]:
import os
import json
from mlflow.models.signature import infer_signature

def log_model_mlflow(run_id, tower):
  with mlflow.start_run(run_id=run_id, nested=True):
    loaded_model = tf.keras.models.load_model(os.path.join(best_model_path, tower))
    loaded_model.summary()

    loaded_model.save(f"{tower}")
    mlflow.log_artifact(f"{tower}", "models/saved_model")

    if tower != "lookup":
#     test_ds = tf.data.TFRecordDataset(filenames=generate_filepaths("splits/test")).take(5).map(read_tfrecord_member_interactions)
#       signature = infer_signature(test_ds.batch(1).as_numpy_iterator().next(), loaded_model.predict(test_ds.batch(1).take(1)))
      mlflow.keras.log_model(loaded_model, f"models/mlflow/{tower}") #, signature=signature)

    # Save plots
    tf.keras.utils.plot_model(loaded_model, to_file=f'{tower}_model.png', show_shapes=True, expand_nested=True, show_dtype=True, dpi=172)
    mlflow.log_artifact(f'{tower}_model.png', "plots/model_architecture/")

log_model_mlflow(run_id, "user")
log_model_mlflow(run_id, "movie")
log_model_mlflow(run_id, "lookup")

WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
Model: "sequential"
_________________________________________________________________
 Layer (type) Output Shape Param # 
=================================================================
 user_id_lookup (StringLooku (None,) 0 
 p) 
 
 embedding (Embedding) (None, 8) 367704 
 
=================================================================
Total params: 367,704
Trainable params: 367,704
Non-trainable params: 0
_________________________________________________________________
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
INFO:tensorflow:Assets written to: user/assets
INFO:tensorflow:Assets written to: user/assets
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
INFO:tensorflow:Assets written to: /tmp/tmpje8icl5s/model/data/model/assets
INFO:tensorflow:Assets written to: /tmp/tmpje8icl5s/model/data/model/assets
WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
Model: "sequential_1"
_________________________________________________________________
 Layer (type) Output Shape Param # 
=================================================================
 title_lookup (StringLookup) (None,) 0 
 
 embedding_1 (Embedding) (None, 8) 13624 
 
=================================================================
Total params: 13,624
Trainable params: 13,624
Non-trainable params: 0
_________________________________________________________________
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
INFO:tensorflow:Assets written to: movie/assets
INFO:tensorflow:Assets written to: movie/assets
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
INFO:tensorflow:Assets written to: /tmp/tmpl1brr1r0/model/data/model/assets
INFO:tensorflow:Assets written to: /tmp/tmpl1brr1r0/model/data/model/assets
WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
WARNING:tensorflow:No training configuration found in save file, so the model was *not* compiled. Compile it manually.
Model: "scann_lookup"
_________________________________________________________________
 Layer (type) Output Shape Param # 
=================================================================
 sequential (Sequential) (None, 8) 367704 
 
=================================================================
Total params: 369,406
Trainable params: 367,704
Non-trainable params: 1,702
_________________________________________________________________
WARNING:absl:Found untraced f

In [0]:
model_name = "sais22_sample_movie_lookup_model"

client = mlflow.tracking.MlflowClient()
result = mlflow.register_model(
    f"runs:/{run_id}/models/mlflow/lookup",
    model_name
)

print("Name: {}".format(result.name))
print("Version: {}".format(result.version))

client.transition_model_version_stage(
    name=model_name,
    version=result.version,
    stage="Staging"
)

Registered model 'sais22_sample_movie_lookup_model' already exists. Creating a new version of this model...
2022/01/04 18:50:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: sais22_sample_movie_lookup_model, version 3
Created version '3' of model 'sais22_sample_movie_lookup_model'.
Name: sais22_sample_movie_lookup_model
Version: 3
Out[92]: <ModelVersion: creation_timestamp=1641322244818, current_stage='Staging', description='', last_updated_timestamp=1641322254429, name='sais22_sample_movie_lookup_model', run_id='7d402175957b482b8b8b373d6b541bd4', run_link='', source='dbfs:/databricks/mlflow-tracking/1776262/7d402175957b482b8b8b373d6b541bd4/artifacts/models/mlflow/lookup', status='READY', status_message='', tags={}, user_id='100012', version='3'>

Promote model to Production

In [0]:
eval_metric = "factorized_top_k/top_10_categorical_accuracy"
ACCEPTABLE_THRESHOLD = 0.1

def get_run_id_version_from_stage(model_name, stage):
  for mv in client.search_model_versions(f"name='{model_name}'"):
        if mv.current_stage == stage:
          return mv.run_id, mv.version
  return None, None

staging_run_id, staging_model_version = get_run_id_version_from_stage(model_name, "Staging")
prod_run_id, prod_model_version = get_run_id_version_from_stage(model_name, "Production")

get_metrics = lambda run_id: mlflow.get_run(run_id).data.metrics['val_factorized_top_k/top_5_categorical_accuracy']

checks_passed = True

if prod_run_id:
  diff = get_metrics(prod_run_id) - get_metrics(staging_run_id)
  print(eval_metric, diff)
  checks_passed &= diff <= ACCEPTABLE_THRESHOLD
  
  if checks_passed:
    client.transition_model_version_stage(
          name=model_name,
          version=prod_model_version,
          stage="Archived"
    )

if checks_passed:
    client.transition_model_version_stage(
        name=model_name,
        version=staging_model_version,
        stage="Production"
      )
print(f"Successfully promoted model version {staging_model_version} to Production")

factorized_top_k/top_10_categorical_accuracy 0.0
Successfully promoted model version 3 to Production

In [0]:
# workaround to load tf saved model instead of mlflow model for custom tf
base_download_path = "/dbfs/tmp/mlflow_download/"
client.download_artifacts(prod_run_id, "models/saved_model/", base_download_path)

Out[114]: '/dbfs/tmp/mlflow_download/models/saved_model/'

Out[132]: <tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f10e13a3430>

In [0]:
def batch_lookup_pandas_udf(batch_df):
  """
  Pandas UDF that loads an index and member model and runs inference based on given UDF

  Parameters
  ----------
  batch_df: A pandas DataFrame

  Returns given dataframe with recommendations and scores
  """
  import numpy as np    
  import tensorflow as tf
  import tensorflow_recommenders as tfrs
  from tensorflow_recommenders.layers.factorized_top_k import ScaNN
  
  lookup_model = tf.saved_model.load(f"{base_download_path}/models/saved_model/lookup")
  dist_batch, styles_batch = lookup_model(batch_df["user_id"])
  batch_df = batch_df.loc[:, ["user_id"]]
  batch_df['recommendations'] = styles_batch.numpy().tolist()
  batch_df['scores'] = dist_batch.numpy().tolist()
  return batch_df

output_schema = StructType([
  StructField("user_id", StringType()),
  StructField("recommendations", ArrayType(StringType())),
  StructField("scores", ArrayType(DoubleType()))
])

Batch Inference with stored model

In [0]:
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql.types import *

batch_size = 1000

# for demo purposes, we will load the test data as inference dataframe
inference_df = spark.read.format("tfrecords").load(f"{data_path}/test").repartition(10).select("user_id").distinct()


batched_df = (inference_df.withColumn("batch", (f.row_number().over(Window.orderBy("user_id"))/f.lit(batch_size)).cast(IntegerType()))
                              .repartition(f.col("batch")))

recs_df = batched_df.groupBy("batch").applyInPandas(batch_lookup_pandas_udf, schema=output_schema).cache()
display(recs_df)

user_id,batch
65784,12
65785,12
65831,12
65832,12
65839,12
65842,12
65845,12
65852,12
6587,12
65891,12


Get the top 50 recs for each member

In [0]:
movie_df = spark.read.format("tfrecords").load(f"{data_path}/movies")

user_recs_df = (recs_df
                  .selectExpr("user_id", "posexplode(recommendations)")
                  .selectExpr("user_id", "pos", "col movie_id")
                  .join(movie_df, ["movie_id"])
                  .select("user_id", "title", "pos")
                  .filter("pos <= 50")
                  .sort("user_id", "pos"))
display(user_recs_df)

user_id,title,pos
10000,Oh Boy (A Coffee in Berlin) (2012),0
10000,When the Cat's Away (Chacun cherche son chat) (1996),1
10000,My Giant (1998),2
10000,Fire on the Mountain (1996),3
10000,"Scarlet Letter, The (1995)",4
10000,Diva (1981),5
10000,"Glass Menagerie, The (1950)",6
10000,"Hundred-Foot Journey, The (2014)",7
10000,"Armstrong Lie, The (2013)",8
10000,To the Arctic (2012),9


Using model with streaming data

In [0]:
SERVING_IP = "ENTER IP HERE"
SERVING_PORT = "ENTER PORT VALUE HERE"

def create_tf_serving_json(data):
  """ Convert the data from dataframe into tf serving json"""
  return {'inputs': {name: value if isinstance(value, list) else [value]
                       for name, value in data.items()} 
                       if isinstance(data, dict) else data.tolist()}


def tfserving_score_model(dataset):
  url = f'http://{SERVING_IP}:{SERVING_PORT}/v1/models/lookup:predict'
  data_json = create_tf_serving_json(dataset)
  response = requests.request(method='POST', url=url, json=data_json)
  if response.status_code != 200:
    raise Exception(f'Request failed with status {response.status_code}, {response.text}')
  return response.json()